In [1]:
from mbmvpa.core.engine import run_mbmvpa
from pathlib import Path

/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/nilearn/glm/__init__.py:56: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  'It may change in any future release of Nilearn.', FutureWarning)


In [4]:
#bids_layout = "tutorial_data/ccsl_prl"
bids_layout = "/data2/project_modelbasedMVPA/PRL"
report_path = "tutorial_report"
Path(report_path).mkdir(exist_ok=True)

In [7]:
def example_adjust(row):
    ## rename data in a row to the name which can match hbayesdm.dd_hyperbolic requirements ##
    if row["outcome"] == 0:
        row["outcome"] = -1
    row["onset"] = row["time_feedback"]
    row["duration"] = row["time_wait"] - row["time_feedback"]
    return row

def example_filter(row):
    return row['choice'] in [1,2]


def example_modulation_dfwise(df_events, param_dict):
    
    df_events = df_events.sort_values(by="onset")
    
    ev = [0,0]
    choices = df_events['choice'].to_numpy()
    outcome = df_events['outcome'].to_numpy()
    modulations = []
    
    eta_pos = float(param_dict['eta_pos'])
    eta_neg = float(param_dict['eta_neg'])
    
    for choice, outcome in zip(choices,outcome):
        choice = int(choice)
        outcome = int(outcome)
        
        PE  =  outcome - ev[choice-1];
        PEnc = -outcome - ev[2 - choice];
        
        modulations.append(PE)
        
        if PE >= 0:
            ev[choice-1] += eta_pos * PE;
            ev[2 - choice] += eta_pos * PEnc;
        else :
            ev[choice-1] += eta_neg * PE;
            ev[2 - choice] += eta_neg * PEnc;
    
    df_events["modulation"] = modulations
    return df_events[['onset','duration','modulation']]


In [8]:
_ = run_mbmvpa(bids_layout=bids_layout,
               mvpa_model='mlp',
               dm_model='prl_fictitious_rp_woa',
               feature_name='zoom2',
               task_name='prl',
               process_name='PEchosen',
               report_path=report_path,
               n_core=4,
               n_thread=4,
               method='5-fold',
               gpu_visible_devices=[2],
              adjust_function= example_adjust,
              filter_function=example_filter,
              latent_function_dfwise=example_modulation_dfwise)

INFO: start loading BIDSLayout
INFO: fMRIPrep is loaded
INFO: MB-MVPA is loaded


0it [00:00, ?it/s]

INFO: 93 file(s) in fMRIPrep.
INFO: existing voxel mask is loaded.: 28179/116380
INFO: start processing 0 fMRI. (nii_img/thread)*(n_thread)=0*4. 93 image(s) is(are) skipped.
INFO: fMRI processing is done.


ValueError: No objects to concatenate